In [16]:
import kmeans, library
from matplotlib import pyplot
import time

In [23]:
t0 = time.time()
pickle_dir = 'C:\\Users\\Colin\\Documents\\SCHOOL_STUFF\\EWU\\W21\\CSCD_530\\Project\\Code\\data_after_removing_words'

t = time.time()
num_docs = 20
vocab, docs = library.load_project(pickle_dir, num_docs, random_files=True)
print(f"loaded {num_docs:,} docs {time.time()-t:.2f}")

t = time.time()
sub_corpus_freqs = kmeans.sub_corpus_frequencies(docs)
print(f"recalc frequencies {time.time()-t:.2f}")  

loaded 20 docs 2.16
recalc frequencies 0.01


# Elbow Method + Silhoette Method

In [18]:
'''
def get_WSSE(cluster_results):
    wsse = 0
    for i in range(len(cluster_results.clusters)):
        for doc in cluster_results.clusters[i]:
            wsse += kmeans.item_distance_dot_product(doc, cluster_results.centroids[i])**2
    return wsse

def get_silhouette(cluster_results):
    # Using equations from
    # https://en.wikipedia.org/wiki/Silhouette_%28clustering%29
    silhouettes = []
    for i in range(len(cluster_results.clusters)):
        cluster_size = len(cluster_results.clusters[i])
        if cluster_size == 1:
            silhouettes.append(0)
        else:
            for j in range(len(cluster_results.clusters[i])):
                doc1 = cluster_results.clusters[i][j]
                # for each pair of documents in the cluster
                for k in range(j+1, len(cluster_results.clusters[i])):
                    doc2 = cluster_results.clusters[i][k]
                    silhouettes.append(kmeans.item_distance_dot_product(doc1,doc2)
    return mean(silhouettes)
'''

def cluster_metrics(cluster_results):
    # Calculates Within Cluster Sum of Square Error (WSSE) to use for Elbow Method and Silhouette value
    # Silhouette source:
    # https://en.wikipedia.org/wiki/Silhouette_%28clustering%29
    # TODO: Update distance metric to be 1/(dot product)
    silhouettes = []
    wsse = 0
    for i in range(len(cluster_results.clusters)):
        # Loop clusters
        cluster_size = len(cluster_results.clusters[i])
        if cluster_size == 1:
            silhouettes.append(0)
        else:
            for j in range(len(cluster_results.clusters[i])):
                # Loop documents in cluster    
                docj = cluster_results.clusters[i][j]
                wsse += kmeans.item_distance_dot_product(doc1, cluster_results.centroids[i])**2
                docj_separation = []
                docj_cohesion = []
                for k in range(len(cluster_results.clusters)):
                    # Loop clusters for each document to get min separation
                    docj_separation.append(kmeans.item_distance_dot_product(docj, cluster_results.centroids[k]))
                # For each pair of documents in the cluster
                for k in range(j+1, len(cluster_results.clusters[i])):
                    # Loop through remaining documents in cluster
                    dock = cluster_results.clusters[i][k]
                    docj_cohesion.append(kmeans.item_distance_dot_product(docj,dock))
                docj_cohesion = (1/(cluster_size-1))*sum(docj_cohesion)
                docj_separation.remove(min(docj_separation))
                docj_separation = min(docj_cohesion)/cluster_size
                silhouettes.append((docj_separation-docj_cohesion)/max(docj_cohesion,docj_separation))
    return wsse, mean(silhouettes)

In [22]:
kmax = 9
K = []
WSSE = []
silhouettes = []

for k in range(kmax):
    K.append(k)
    results = kmeans.find_clusters(docs, k)
    wsse, silhouette = cluster_metrics(results)
    WSSE.append(wsse)
    silhouettes.append(wsse)

IndexError: list index out of range